In [ ]:
import torch
import numpy as np
import rasterio
from rasterio.features import shapes
import geopandas as gpd
from shapely.geometry import shape
import os
from model import UNet  # Make sure UNet class is available in model.py or earlier notebook

In [ ]:
def load_model(weights_path):
    model = UNet(in_channels=3, out_classes=3)
    model.load_state_dict(torch.load(weights_path, map_location='cpu'))
    model.eval()
    return model


In [ ]:
def predict_mask(model, image_path):
    with rasterio.open(image_path) as src:
        image = src.read().astype(np.float32) / 1.0
        meta = src.meta.copy()

    x = torch.tensor(image).unsqueeze(0)  # (1, 3, H, W)
    with torch.no_grad():
        out = model(x)
        pred = out.argmax(dim=1).squeeze().cpu().numpy().astype(np.uint8)

    return pred, meta


In [ ]:
def save_mask(mask, meta, output_path):
    meta.update(dtype='uint8', count=1)
    with rasterio.open(output_path, 'w', **meta) as dst:
        dst.write(mask, 1)


In [ ]:
def mask_to_shapefile(mask_path, output_shapefile_path):
    with rasterio.open(mask_path) as src:
        mask = src.read(1)
        transform = src.transform

    geometries = list(shapes(mask, mask > 0, transform=transform))
    records = []
    for geom, val in geometries:
        records.append({"geometry": shape(geom), "class": int(val)})

    gdf = gpd.GeoDataFrame.from_records(records, crs=src.crs)
    gdf.to_file(output_shapefile_path)

    print(f"✅ Shapefile saved to {output_shapefile_path}")

In [ ]:
input_image = "data/test/R2F_TEST_01.tif"
output_mask = "outputs/inference/R2F_TEST_01_predicted_mask.tif"
output_shapefile = "outputs/inference/R2F_TEST_01_shapes.shp"
weights_path = "model/unet_cloud_shadow.pth"

os.makedirs("outputs/inference", exist_ok=True)

model = load_model(weights_path)
pred_mask, meta = predict_mask(model, input_image)
save_mask(pred_mask, meta, output_mask)
mask_to_shapefile(output_mask, output_shapefile)